In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import keras

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [ ]:
allData = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nba data/Full_dataset4.csv')

In [ ]:
for i in range(len(allData)):
    if(allData.iloc[i,22]=='L'):
        allData.iloc[i,22] = 0
    else:
        allData.iloc[i,22] = 1

## Dataset A (all data)

In [ ]:
X_A , y = allData.values[:, :-1], allData.values[:,-1]

## Dataset B (cfs subset - bestFirst on training set)

In [ ]:
Bvalues = ['FG_PCT_HOME','FG3_PCT_HOME','DREB_HOME','AST_HOME','STL_HOME',
          'FG_PCT_AWAY','FG3_PCT_AWAY','FT_PCT_AWAY','DREB_AWAY','AST_AWAY','PF_AWAY',
          'WL_HOME']

X_B, Y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nba data/Full_dataset4.csv',
                     usecols = Bvalues).values[:, :-1].astype(np.float32), allData.values[:,-1].astype(np.float32)

## Dataset C ( GreeedyStepwise on training set)

In [ ]:
Cvalues = ['FG_PCT_HOME','FG3_PCT_HOME','FT_PCT_HOME','DREB_HOME','AST_HOME','STL_HOME',
          'FG_PCT_AWAY','FG3_PCT_AWAY','FT_PCT_AWAY','DREB_AWAY','AST_AWAY','PF_AWAY',
          'WL_HOME']

X_C, Y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nba data/Full_dataset4.csv',
                     usecols = Cvalues).values[:, :-1].astype(np.float32), allData.values[:,-1].astype(np.float32)

## Dataset D (Shooting 4 values)

In [ ]:
Dvalues = ['FG_PCT_HOME','FT_PCT_HOME',
          'FG_PCT_AWAY','FT_PCT_AWAY',
          'WL_HOME']

X_D, Y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nba data/Full_dataset4.csv',
                     usecols = Dvalues).values[:, :-1].astype(np.float32), allData.values[:,-1].astype(np.float32)

## Dataset E (Shooting 6 values)

In [ ]:
Evalues = ['FG_PCT_HOME','FT_PCT_HOME','FG3_PCT_HOME',
          'FG_PCT_AWAY','FT_PCT_AWAY','FG3_PCT_AWAY',
          'WL_HOME']

X_E, Y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nba data/Full_dataset4.csv',
                     usecols = Evalues).values[:, :-1].astype(np.float32), allData.values[:,-1].astype(np.float32)

## Dataset F (personal  belief)

In [ ]:
Fvalues = ['FG_PCT_HOME','DREB_HOME','AST_HOME','STL_HOME',
          'FG_PCT_AWAY','DREB_AWAY','AST_AWAY','STL_AWAY',
          'WL_HOME']

X_F, Y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nba data/Full_dataset4.csv',
                     usecols = Fvalues).values[:, :-1].astype(np.float32), allData.values[:,-1].astype(np.float32)

## Dataset G (GReedy on test set)


In [ ]:
Gvalues = ['FG_PCT_HOME','FT_PCT_HOME','OREB_HOME','BLK_HOME',
          'FG_PCT_AWAY','DREB_AWAY',
          'WL_HOME']

X_G, Y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nba data/Full_dataset4.csv',
                     usecols = Gvalues).values[:, :-1].astype(np.float32), allData.values[:,-1].astype(np.float32)

## Dataset H (BestFirst on test set)

In [ ]:
Hvalues = ['FG_PCT_HOME','FG3_PCT_HOME','FT_PCT_HOME','OREB_HOME','REB_HOME','STL_HOME','BLK_HOME','TOV_HOME',
           'FG_PCT_AWAY','FG3_PCT_AWAY','FT_PCT_AWAY','DREB_AWAY','STL_AWAY','BLK_AWAY',
           'WL_HOME']

X_H, Y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nba data/Full_dataset4.csv',
                     usecols = Hvalues).values[:, :-1].astype(np.float32), allData.values[:,-1].astype(np.float32)

In [ ]:
def standardise(x):
    #Standardize the data
    data_scaler = StandardScaler()
    data_scaler.fit(x)
    x = data_scaler.transform(x)
    return x, data_scaler


# def split_data_and_standardise(x,y,train_size=30000):
    
#     x = standerdise(x)
    
#     x_train = x[:train_size]
#     y_train = y[:train_size]

#     x_test = x[train_size:]
#     y_test = y[train_size:]
    
    
    
#     return x_train, y_train, x_test, y_test

## Spliting data 

In [ ]:
# x_train, y_train, x_test, y_test = split_data_and_standerdise(X_A,Y,30000)
# X_A = standardise(X_A)

In [ ]:
x_train = X_B
y_train = Y
value = Bvalues
x_train, sca = standardise(x_train)
n_features = x_train.shape[1]
n_features

11

## Creating/ Running model

In [ ]:
import tensorflow as tf
from keras import Sequential, Input
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Input(shape=(n_features,)))
model.add(Dense(200,activation='relu',
                     kernel_initializer='he_normal'))
model.add(Dense(100,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(50,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(25,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(12,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(6,activation='relu',kernel_initializer='he_normal'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="auto",
    restore_best_weights=True,
)


In [ ]:
model.fit(x_train,y_train,
          shuffle=True,
          validation_split=0.3,
          epochs=50,
          callbacks=[callback])

Epoch 1/50
811/811 [==============================] - 3s 2ms/step - loss: 0.3674 - accuracy: 0.8286 - val_loss: 0.3530 - val_accuracy: 0.8409
Epoch 2/50
811/811 [==============================] - 2s 2ms/step - loss: 0.3288 - accuracy: 0.8524 - val_loss: 0.3554 - val_accuracy: 0.8420
Epoch 3/50
811/811 [==============================] - 2s 2ms/step - loss: 0.3241 - accuracy: 0.8548 - val_loss: 0.3644 - val_accuracy: 0.8295
Epoch 4/50
811/811 [==============================] - 2s 2ms/step - loss: 0.3163 - accuracy: 0.8566 - val_loss: 0.3775 - val_accuracy: 0.8260
Epoch 5/50
811/811 [==============================] - 2s 2ms/step - loss: 0.3199 - accuracy: 0.8566 - val_loss: 0.3656 - val_accuracy: 0.8296
Epoch 6/50
811/811 [==============================] - 2s 2ms/step - loss: 0.3174 - accuracy: 0.8554 - val_loss: 0.3621 - val_accuracy: 0.8362


In [ ]:
# FUNCTION TO EVALUATE MODEL
def game_and_accuracy(predictions, true):
  correct = 0
  for i in range(len(predictions)):
      print('Game', i,':')
      value = round(predictions[i][0])
      camp = ('home' if value == 1 else 'away' )
      if(value != int(true[i])):
          print('Missed, chose '+ camp)
      else:
          print('gotit, chose ' +camp)
          correct = correct + 1
  accuracy = correct/len(predictions)
  print(accuracy)


In [ ]:
TestingDF = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nba data/nba050521_edit.csv' , usecols = value)

In [ ]:
xTest, yTest = TestingDF.values[:, :-1], TestingDF.values[:,-1]

In [ ]:
xTest = sca.transform(xTest)
p = model.predict(xTest)
game_and_accuracy(p,yTest)
model.evaluate(xTest, yTest)

Game 0 :
gotit, chose away
Game 1 :
Missed, chose away
Game 2 :
Missed, chose home
Game 3 :
gotit, chose home
Game 4 :
gotit, chose home
Game 5 :
Missed, chose home
Game 6 :
Missed, chose away
Game 7 :
Missed, chose home
0.375
1/1 [==============================] - 0s 14ms/step - loss: 0.8136 - accuracy: 0.3750


[0.8136452436447144, 0.375]

In [58]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 200)               2400      
_________________________________________________________________
dense_22 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_23 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_24 (Dense)             (None, 25)                1275      
_________________________________________________________________
dense_25 (Dense)             (None, 12)                312       
_________________________________________________________________
dense_26 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_27 (Dense)             (None, 1)                